In [13]:
%load_ext autoreload
%autoreload 2
import os
import pystan
import numpy as np
import scipy as sp
from __future__ import division
# import ite
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
import re
import glob
import tensorflow as tf
from tensorflow.contrib import slim
ds = tf.contrib.distributions
use_entropy=True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
#gmm set up
param_dim = 2
num_mixtures =8
radius = 0.5
thetas = np.linspace(0, 2 * np.pi, num_mixtures + 1)[:num_mixtures]
xs, ys = radius * np.sin(thetas), radius * np.cos(thetas)
means_x = list([np.array([a,b]) for a,b in zip(xs,ys)])
std = 0.05
u_mean=tuple(zip(xs, ys))
u_cov=tuple([(std, std)] * num_mixtures)
def gmm(x):
        ds = tf.contrib.distributions
        gauss_ref = ds.Mixture(
          cat=ds.Categorical(probs=[1.0/8]*8),
          components=[
            ds.MultivariateNormalDiag(loc=u_mean[0], scale_diag=u_cov[0]),
            ds.MultivariateNormalDiag(loc=u_mean[1], scale_diag=u_cov[1]),
            ds.MultivariateNormalDiag(loc=u_mean[2], scale_diag=u_cov[2]),
            ds.MultivariateNormalDiag(loc=u_mean[3], scale_diag=u_cov[3]),
            ds.MultivariateNormalDiag(loc=u_mean[4], scale_diag=u_cov[4]),
            ds.MultivariateNormalDiag(loc=u_mean[5], scale_diag=u_cov[5]),
            ds.MultivariateNormalDiag(loc=u_mean[6], scale_diag=u_cov[6]),
            ds.MultivariateNormalDiag(loc=u_mean[7], scale_diag=u_cov[7]),
              
        ])      
        return gauss_ref.log_prob(x)
def w1(z):
    return tf.sin(2.*np.pi*z[0]/4.)
def w2(z):
    return 3.*tf.exp(-.5*(((z[0]-1.)/.6))**2)
def w3(z):
    return 3.*(1+tf.exp(-(z[0]-1.)/.3))**-1
def pot1(z):
    z = tf.transpose(z)
    ans=.5*((tf.norm(z, ord=2, axis=0)-2.)/.4)**2 - tf.log(tf.exp(-.5*((z[0]-2.)/.6)**2) + tf.exp(-.5*((z[0]+2.)/.6)**2))
    return -ans
def pot2(z):
    z = tf.transpose(z)
    ans=.5*((z[1]-w1(z))/.4)**2
    return -ans
def pot3(z):
    z = tf.transpose(z)
    ans=-tf.log(tf.exp(-.5*((z[1]-w1(z))/.35)**2) + tf.exp(-.5*((z[1]-w1(z)+w2(z))/.35)**2))
    return -ans
def pot4(z):
    z = tf.transpose(z)
    ans=-tf.log(tf.exp(-.5*((z[1]-w1(z))/.4)**2) + tf.exp(-.5*((z[1]-w1(z)+w3(z))/.35)**2))
    return -ans

In [15]:
hidden_size=256
def linear(input, output_dim, scope='linear', stddev=0.01,use_SN=False, update_collection=None):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope):
        w = tf.get_variable('weights', [input.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('biases', [output_dim], initializer=const)
        if use_SN:
            print('using SN')
            return tf.matmul(input,spectral_normed_weight(w, update_collection=update_collection))+b
        else:
            return tf.matmul(input, w) + b
def posterior(eps,reuse=False):
    with tf.variable_scope('posterior',reuse=reuse) as scope:
        h=tf.nn.relu(linear(eps,hidden_size,'p_h1'))
        h=tf.nn.relu(linear(h,hidden_size,'p_h2'))
        out=linear(h,2,scope='p_out')
        return out
def adversary(z,reuse=False):
    with tf.variable_scope('adversary',reuse=reuse) as scope:
        h=tf.nn.relu(linear(z,hidden_size,'a_h1'))
        out=linear(h,1,scope='a_out')
        return out
def inference(input,reuse=False):
    with tf.variable_scope('inference',reuse=reuse) as scope:
        h=tf.nn.relu(linear(input,hidden_size,'i_h1'))
        h=tf.nn.relu(linear(h,hidden_size,'i_h2'))
        out=linear(h,2,scope='i_out')
        return out
    

In [16]:
batch_size=1024
is_adapt_contrast=True
get_logprob=gmm
tf.reset_default_graph()
z0=tf.random_normal([batch_size,param_dim],name='z0')
eps=tf.random_normal([batch_size,param_dim],name='eps')
z_=posterior(eps)

beta=tf.constant(1.0)
if is_adapt_contrast:
    Ez_, Varz_ = tf.nn.moments(z_, [0], keep_dims=True)
    stdz_ = tf.sqrt(Varz_) + 1e-6
    Ez_ = tf.stop_gradient(Ez_)
    stdz_ = tf.stop_gradient(stdz_)

    zr = Ez_ + stdz_ * z0
    znorm_ = (z_ - Ez_) / stdz_
gauss_ref=ds.MultivariateNormalDiag(loc=[0.,0.],scale_diag=[1.0,1.0])

if is_adapt_contrast:
    print('using AC')
    logr_=gauss_ref.log_prob(znorm_)
    Ti=adversary(z0)
    Td=adversary(znorm_,reuse=True)
else:
    logr_=gauss_ref.log_prob(z_)
    Ti=adversary(z0)
    Td=adversary(z_,reuse=True)
    
logprob=get_logprob(z_)
mean_logprob=tf.reduce_mean(logprob)
loss_primal=tf.reduce_mean(logr_+Td-logprob)

d_loss_d=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Td,labels=tf.ones_like(Td)))
d_loss_i=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Ti,labels=tf.zeros_like(Ti)))
loss_dual=d_loss_i+d_loss_d

entropy_term=beta*mean_logprob


if use_entropy:
    loss_primal+=1*entropy_term


using AC


In [17]:
pvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "posterior")
dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "adversary")
# ivars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'inference')
popt=tf.train.AdamOptimizer(2e-4,beta1=0.5)
dopt=tf.train.AdamOptimizer(2e-4,beta1=0.5)
grads_primal =  popt.compute_gradients(loss_primal, var_list=pvars)
grads_dual = dopt.compute_gradients(loss_dual, var_list=dvars)

train_primal = popt.apply_gradients(grads_primal)
train_dual = dopt.apply_gradients(grads_dual)

train_step = [train_primal, train_dual]

In [18]:
# Parameters for visualization
idx0, idx1 = 0, 1
labels = [r'$\mu$', r'$\tau$', r'$\eta_1$']
xlabel = labels[idx0]
ylabel = labels[idx1]

plt.rc('font', size=16)
def get_samples(sess, nbatches=10):
    zs = np.zeros([nbatches, batch_size, param_dim])
    for i in range(nbatches):
        zs[i] = sess.run(z_)
    zs = zs.reshape(-1, param_dim)
    return zs
def scatter(mu, tau,savepath, bbox=[-5, 5, -5, 5], xlabel="", ylabel=""):
    fig, ax = plt.subplots()
    ax.axis(bbox)
    ax.set_aspect(1.0*4/5*abs(bbox[1]-bbox[0])/abs(bbox[3]-bbox[2]))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
                        
    ax.scatter(mu, tau, edgecolor='b', alpha=0.1)
    plt.plot(xs,ys,'ro')
#     plt.show()
    plt.savefig(savepath)
    
def plot_results(save_path):
    q = get_samples(sess, nbatches=10)
    q1 = q[:, idx0]
    q2 = q[:, idx1]
    bbox = [-0.95,0.95,-0.95,0.95]
#     bbox=[-5, 5, -5, 5]
    scatter(q1, q2,save_path,bbox,xlabel=xlabel, ylabel=ylabel)

In [19]:
from tqdm import tqdm_notebook, tnrange

def run_training(sess, niter=10000, betas=None, npretrain=0):    
    outdir='./ce_8gmm_%.01f_%.02f'%(radius,std)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    if betas is None:
        betas = []
        
    for i in tnrange(npretrain, desc="Pretrain"):
        sess.run(train_dual)

    pbar = tnrange(niter+1, desc="Train")
    for i in pbar:
        if i >= np.size(betas):
            beta_i = 1.
        else:
            beta_i = betas[i]
                           
        _, lp, ld= sess.run(
                [train_primal, loss_primal, loss_dual],
                feed_dict={beta: beta_i}
        )
        
        sess.run(train_dual)
#         sess.run(train_dual)
        if i%10000==0:
            
            samples=get_samples(sess,nbatches=20)
            if use_entropy:
                plot_results(os.path.join(outdir,'EN_%08d.png'%i))
                np.save(os.path.join(outdir,'EN_%08d'%i),samples)
            else:
                plot_results(os.path.join(outdir,'%08d.png'%i))
                np.save(os.path.join(outdir,'%08d'%i),samples)
        pbar.set_description("lp=%.3f, ld=%.3f"  % (lp, ld))



In [ ]:
try:
    sess.close()
except NameError:
    pass
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
iter_max = 300000
# AVB
# run_training(sess, niter=iter_max,npretrain=0)
run_training(sess, niter=iter_max,betas=np.linspace(0.9,0.0,iter_max-10000),npretrain=0)

HBox(children=(IntProgress(value=1, bar_style=u'info', description=u'Pretrain: ', max=1), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Train: ', max=300001), HTML(value=u'')))

##### 